In [1]:
from huggingface_hub import login
import os
from dotenv import load_dotenv

load_dotenv()
hf_token = os.getenv("HUGGING_FACE_TOKEN")
login(token=hf_token)

In [2]:
from perplexity import Perplexity
from dotenv import load_dotenv

load_dotenv() 

client = Perplexity() # PERPLEXITY_API_KEY is read from environment variable

search = client.search.create(
    query=[
      "What is Comet Browser?",
      "Perplexity AI",
      "Perplexity Changelog"
    ]
)

for result in search.results:
    print(f"{result.title}: {result.url}")

Introducing Comet: Browse at the speed of thought: https://www.perplexity.ai/hub/blog/introducing-comet
Browse at the speed of thought: https://www.perplexity.ai/comet/
Comet Browser Review 2025: Features, Pricing, Pros & Cons: https://efficient.app/apps/comet
Comet Browser: The Revolutionary AI-Powered ...: https://dev.to/s_chathuranga_j/comet-browser-the-revolutionary-ai-powered-browser-that-actually-does-your-work-n6d
Perplexity's AI-Native Browser Comet is HERE: https://www.youtube.com/watch?v=ri_bFrDp44M
What Is Comet and Why Everyone's Talking About It - Apidog: https://apidog.com/blog/what-is-comet/
10 Comet Browser Features That Make Chrome & Edge Look ANCIENT: https://www.youtube.com/watch?v=BfC_Lk3NVSs
Perplexity launches Comet, an AI-powered web browser: https://techcrunch.com/2025/07/09/perplexity-launches-comet-an-ai-powered-web-browser/
Comet: The AI browser gets personal: https://www.ibm.com/think/news/comet-perplexity-take-agentic-browser
Comet vs Chrome: Can AI Browser

In [3]:
import json
from pprint import pprint

goal = "Analyze APPL stock for a potential investment."

prompt = f"""
    As an autonomous investment research agent, create a detailed plan to achieve this goal:
    Goal: {goal}
    
    Available tools: web search, financial data API, news aggregator, sentiment analysis tool

    Create a plan with:
    1. Logical research steps (3-7 steps)
    2. Estimated time in minutes
    3. Required tools for each step
    4. Success criteria to evaluate completion
    
    Answer in JSON format:
    {{
        "goal": "...",
        "steps": ["step1", "step2", ...],
        "estimated_time": 30,
        "required_tools": ["tool1", "tool2", ...],
        "success_criteria": ["criteria1", "criteria2", ...]
    }}
"""


response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="sonar",
)


### Presenting the data generated

In [4]:

#print("Raw Response:\n", response.choices[0].message.content)
plan_data = json.loads(response.choices[0].message.content)

pprint(plan_data)

{'estimated_time': 45,
 'goal': 'Analyze AAPL stock for a potential investment',
 'required_tools': ['financial data API',
                    'technical charting software or financial data API',
                    'news aggregator',
                    'sentiment analysis tool',
                    'web search'],
 'steps': ['Collect up-to-date fundamental financial data for Apple Inc. '
           'including recent earnings, revenue, and sales metrics to '
           'understand the company’s financial health.',
           'Perform a technical analysis of AAPL’s stock price trends and '
           'patterns using charts, support/resistance levels, and volume '
           'indicators to assess price momentum and potential entry points.',
           'Aggregate and analyze recent news articles and market sentiment '
           'around Apple, focusing on factors like regulatory developments, '
           'product launches, and geopolitical risks.',
           'Use sentiment analysis tool

### Sentiment analysis using perplexity API

#### Get information from FinTools

In [ ]:
from src.ingestion import FinancialDataIngestion, NewsDataIngestion

ticker = "PLTR"  # Example stock ticker
stock_info = FinancialDataIngestion().execute(ticker)
pprint(stock_info)

news_articles = NewsDataIngestion().execute(ticker, 5)
for article in news_articles:
    pprint(article)


{'data': {'52_week_high': 190.0,
          '52_week_low': 39.015,
          'avg_volume_30d': 56519391.86363637,
          'beta': 2.595,
          'company_summary': 'Palantir Technologies Inc. builds and deploys '
                             'software platforms for the intelligence '
                             'community to assist in counterterrorism '
                             'investigations and operations in the United '
                             'States, the United Kingdom, and internationally. '
                             'It provides Palantir Gotham, a software '
                             'platform, which enables users to identify '
                             'patterns hidden deep within datasets, ranging '
                             'from signals intelligence sources to reports '
                             'from confidential informants, as well as '
                             'facilitates the hand-off between analysts and '
                             'o

#### Generate sentiment analysis and confidence score

In [6]:

goal = f"Analyze {ticker} stock for a potential investment."

prompt = f"""
    As an autonomous investment research agent, create short sentiment analysis
      about the stock based on the context and recent news.
      Describe the sentiment as Positive, Negative, or Neutral.
      Give a summary of your analysis, and a confidence score between 0 and 1.
    Goal: {goal}

    Context: {stock_info}

    Recent News: {news_articles}

    Answer in JSON format:
    {{
        "stock": "...",
        "sentiment": "Positive",
        "summary": "...",
        "confidence": 0.85
    }}
"""


response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="sonar",
)

print("Raw Response:\n", response.choices[0].message.content)

Raw Response:
 ```json
{
    "stock": "PLTR",
    "sentiment": "Positive",
    "summary": "Palantir Technologies (PLTR) has shown a strong performance recently, with its stock reaching all-time highs following a significant earnings beat in Q2 2025. The company's AI surge has ignited substantial gains, and its Foundry platform has driven a significant increase in U.S. commercial revenues. However, concerns about valuation and the AI bubble's sustainability may temper long-term optimism.",
    "confidence": 0.85
}
```


## Adding Memory to the planner operation as an example

### 1. add memories

In [7]:
from src.memory import MemoryWorker

# adding memories as data example
memory_worker = MemoryWorker()
memory_worker.execute('add', 'AAPL Q2 earnings beat expectations', ['AAPL', 'earnings'])
memory_worker.execute('add', 'AAPL announced a new AI partnership with NASA', ['AAPL', 'AI'])
memory_worker.execute('add', 'TSLA stock is volatile due to production concerns', ['TSLA', 'production'])

print("3 memories added.")

3 memories added.


### 2. ingest data

In [8]:
from src.memory import MemoryWorker


### Search Memories by tag

ticker = "AAPL"  # Example stock ticker
stock_info = FinancialDataIngestion().execute(ticker)
news_articles = NewsDataIngestion().execute(ticker, 5)
memories = memory_worker.execute('search', 'AAPL')


### Prompting for analysis

It demonstrates how the memory impacts the results. 
You can see that the added memories will appear on the summary

In [9]:

goal = f"Analyze {ticker} stock for a potential investment."

prompt = f"""
    As an autonomous investment research agent, create short sentiment analysis
      about the stock based on the context and recent news.
      Describe the sentiment as Positive, Negative, or Neutral.
      Give a summary of your analysis, and a confidence score between 0 and 1.
    Goal: {goal}

    Context: {stock_info}

    Recent News: {news_articles}

    Relevant Memories: {memories}

    Answer in JSON format:
    {{
        "stock": "...",
        "sentiment": "Positive",
        "summary": "...",
        "confidence": 0.85
    }}
"""


response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="sonar",
)

print("Raw Response:\n", response.choices[0].message.content)

Raw Response:
 {
  "stock": "AAPL",
  "sentiment": "Positive",
  "summary": "Apple's stock price is near its 52-week high, reflecting strong market confidence with a current price of about $256.26. Recent quarterly earnings have beaten expectations, signaling solid financial performance. The company also secured a new AI partnership with NASA, indicating innovation and growth potential. Although the stock experienced a minor price dip of 0.17%, trading volume is below its 30-day average, suggesting limited selling pressure. Market valuation indicators such as a P/E ratio around 30.8 remain reasonable for a technology giant with strong returns on assets and equity. Additionally, notable investor interest is evidenced by positive news sentiment and strategic developments in emerging tech sectors like AI and hardware. Overall, these factors imply bullish outlook for AAPL as a potential investment, balanced with typical tech sector volatility.",
  "confidence": 0.9
}
